In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [4]:
ls data

men_shoes.csv


In [5]:
df=pd.read_csv('data/men_shoes.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df=pd.read_csv('data/men_shoes.csv',low_memory=False)

In [0]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
  x=df[feats].values
  y=df['prices_amountmin'].values
  scores=cross_val_score(model,x,y,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat']=df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [9]:
run_model (['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [10]:
model=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [11]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [12]:
test={'key':'value'}
test['key']
str(test)

"{'key': 'value'}"

In [13]:
str_dict='[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  output_dict={}
  if str(x)=='nan':return output_dict

  features= literal_eval(x.replace('\\"','"'))
  for item in features:
    key=item['key'].lower().strip()
    value=item['value'][0].lower().strip()

    output_dict[key]=value
  return output_dict
df['features_parsed']=df['features'].map(parse_features)

In [16]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [39]:
keys=set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [40]:
def get_name_feat(key):
  return 'feat_'+key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)]=df.features_parsed.map(lambda feats: feats[key] if key  in feats else np.nan)

In [21]:
df[df['feat_athlete'].isnull()].shape

(18272, 526)

In [0]:
keys_stat={}
for key in keys:
  keys_stat[key] = df[False==df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100

In [24]:
{k:v for k,v in keys_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat']=df['feat_brand'].factorize()[0]
df['feat_color_cat']=df['feat_color'].factorize()[0]
df['feat_gender_cat']=df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat']=df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat']=df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]
df['feat_metal type_cat'] = df['feat_metal type'].factorize()[0]
df['feat_shape_cat'] = df['feat_shape'].factorize()[0]


In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )

In [0]:
feats=['brand_cat','feat_brand_cat','feat_metal type_cat','feat_shape_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_sport_cat']
model=RandomForestRegressor(max_depth=5, n_estimators=100)

result = run_model(feats,model)

In [0]:
x=df[feats].values
y=df['prices_amountmin'].values

m=RandomForestRegressor(max_depth=5, n_estimators=100,)

In [58]:
x=df[feats].values
y=df['prices_amountmin'].values

m=RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
m.fit(x,y)

print(result)
perm=PermutationImportance(m, random_state=1).fit(x,y);
eli5.show_weights(perm, feature_names=feats)

(-57.270857242414706, 4.2322257797637475)


Weight,Feature
0.2562 ± 0.0077,brand_cat
0.1035 ± 0.0075,feat_material_cat
0.0253 ± 0.0026,feat_gender_cat
0.0178 ± 0.0008,feat_brand_cat
0.0126 ± 0.0006,feat_shape_cat
0.0089 ± 0.0012,feat_metal type_cat
0.0031 ± 0.0011,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [41]:
df['brand'].value_counts()

Nike                 1709
PUMA                  574
Ralph Lauren          524
New Balance           361
VANS                  356
                     ... 
Party Favors Plus       1
David Spencer           1
NBA                     1
Spiewak                 1
Vans Pro                1
Name: brand, Length: 1856, dtype: int64

In [45]:
df[df['brand']=="Nike"].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [60]:
!git add day5.ipynb

fatal: pathspec 'day5.ipynb' did not match any files
